# Project Introduction

__[Department of Transportation's Responsibilities](https://www.google.com/search?client=safari&rls=en&q=department+of+transportation+responsibilities&ie=UTF-8&oe=UTF-8)__

Through this dataset, I have identified the three following patterns:
1. IDEA 1 w/ LOCAL LINK
2. IDEA 2 w/ LOCAL LINK
3. IDEA 3 w/ LOCAL LINK

## Resources:

- __[Kaggle-Full](https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents/data)__

- __[Kaggle-Sampled](https://drive.google.com/file/d/1U3u8QYzLjnEaSurtZfSAS_oh9AT2Mn8X/edit)__

- __[Bing API](https://learn.microsoft.com/en-us/bingmaps/rest-services/traffic/get-traffic-incidents#supported-http-methods)__

- __[MQuest API](https://developer.mapquest.com/documentation/api/traffic/incidents/get.html)__

- __['A Countrywide Traffic Accident Dataset'](https://arxiv.org/pdf/1906.05409)__

- __['Accident Risk Prediction based on Heterogenous Sparse Data: New Dataset & Insights](https://arxiv.org/pdf/1909.09638)__

# 1-Setup Environment

## Libraries

In [1]:
#Utilities
import warnings

# Data Basics
import pandas as pd
import numpy as np
import missingno as msgno

#Time Data
from datetime import date, timedelta


#PySpark
from pyspark.sql import SparkSession
from pyspark.mllib.stat import Statistics
import pyspark.sql.functions as F

#Visuals
import matplotlib.pyplot as plt
import seaborn as sns


# Statistical Analysis
from scipy.stats import zscore, f_oneway, chi2_contingency, chisquare
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Spatial Tools
import geopy
import geopy.distance

# Text Tools
import re
from wordcloud import WordCloud

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.util import ngrams

In [2]:
#api = geopy.Nominatim(user_agent='Capstone_Project_1')
#(a,b) = data[data['Street'].str.startswith('I-',na=False)].iloc[0][['Start_Lat','Start_Lng']]
#lat_long = f'{a},{b}'
#local = api.reverse(lat_long)
#local.address

## Info on New Libraries
For improved analysis, these libraries were included, but not covered in the material:

- __[Geopy:  ](https://geopy.readthedocs.io/en/stable/)__

- MissingNo:  
    * __[Library](https://github.com/ResidentMario/missingno)__
    
    * __[Tutorial](https://www.geeksforgeeks.org/python-visualize-missing-values-nan-values-using-missingno-library/)__

## Settings

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)
_LITE_SWITCH_ = True
_SPARK_ = False

## Custom Functions

In [4]:
# For High-level data exploration
def count_outliers(df_col,cap=3):
    zs = zscore(df_col)
    return df_col[zs > cap].shape[0]

In [5]:
# For Identify Unusual Weather Pattern(s)
def unusual_weather(observation):
    flag = False
    return flag

In [6]:
def dist(pt_0,pt_1):
    return geopy.distance.distance(pt_0,pt_1).miles

In [ ]:
def highway_indicator(eventA):
    pass

In [8]:
def same_road(eventA,eventB):
    indicator = False
    road_identifiers = ['Street','City','State','Zipcode']
    indicator = all(eventA[id]==eventB[id] for id in road_identifiers)
    return indicator

In [9]:
# For indicating whether two events are close in space and, optionally, time.
def proximity_indicator(eventA,eventB,space_band=15,time_band=0):
    flag = False
    distance = dist(eventA[['longitude','latitude']],eventB[['longitude','latitude']])
    if (time_band > 0):
        time_lapse = abs(eventA['time']-eventB['time'])
        flag = bool((space_band > distance) and (time_band > time_lapse))
    else:
        flag = bool(space_band > distance)
    return flag

In [ ]:
def test_category(feature_name):
    # Time of Impact--ANOVA
    model_time = ols(f"Q('Time_of_Impact(hr)') ~ C({feature_name})", data=data_clean).fit()
    anova_table_time = sm.stats.anova_lm(model_weather, typ=2)
    # Distance of Impact--ANOVA
    model_distance = ols(f"Q('Distance(mi)') ~ C({feature_name})", data=data_clean).fit()
    anova_table_distance = sm.stats.anova_lm(model_weather, typ=2)
    # Severity Distribution--ANOVA
    contingency_table = pd.crosstab(data_clean['Weather_Condition'], data_clean['Severity'])
    chi_results = chi2_contingency(contingency_table)
    return anova_table_time, anova_table_distance, chi_results

In [ ]:
# Holiday Indicators
# Given by Google:
def calculate_memorial_day(year):
    # Get the last day of May
    last_day_of_may = date(year, 5, 31)

    # Get the weekday of the last day of May (0=Monday, 6=Sunday)
    weekday = last_day_of_may.weekday()

    # Calculate the offset to get to the last Monday
    offset = (weekday - 0) % 7

    # Calculate Memorial Day
    memorial_day = last_day_of_may - timedelta(days=offset)

    return memorial_day

# Given by Google:
def get_thanksgiving_date(year):
    # Get the first day of November
    first_november = date(year, 11, 1)
    
    # Calculate the day of the week for the first of November (0 = Monday, 6 = Sunday)
    first_day_weekday = first_november.weekday()
    
    # Calculate the number of days to add to get to the first Thursday
    days_to_first_thursday = (3 - first_day_weekday) % 7
    
    # Calculate the date of the first Thursday of November
    first_thursday = first_november + timedelta(days=days_to_first_thursday)
    
    # Add 3 weeks (21 days) to get to the fourth Thursday (Thanksgiving)
    thanksgiving_date = first_thursday + timedelta(days=21)
    
    return thanksgiving_date

def get_labor_day(year):
    first = date(year,9,1)
    d = first
    while d.weekday() > 0:
       d = d + timedelta(days=1)
    return d

## Load Dataset

In [ ]:
if _SPARK_:
    spark = SparkSession.builder.appName("Accident Data Project").getOrCreate()
    data = spark.read.csv('US_Accidents_March23.csv',header=True,inferSchema=True)
else:
    if _LITE_SWITCH_:
        data = pd.read_csv('US_Accidents_March23_sampled_500k.csv')
    else:
        data = pd.read_csv('US_Accidents_March23.csv')
    

# 2-Initial EDA

## Schema & Feature Basics

In [ ]:
if _SPARK_:
    data.printSchema()
    print("Features: ",len(data.columns))
    print("Entries:  ",data.count())
else:
    data.info()

In [ ]:
if _SPARK_:
    data.show(5,vertical=False)
else:
    data.head(5)

In [ ]:
if _SPARK_:
    data.describe().show()
else:
    pass
    

Very large dataset with over 7.7 million observations.  (I also chose to do some initial analysis ont he kaggle-provided sampled dataset which contains only 500k observations.)  So we will have to handle with care either by using Spark or through other tricks.


Notes on Features: Dimensionally much more managable with just 45 features.
- 'Severity' would appear to be the primary variable of interest; 'Description' might be interesting to experiment with.
    * Upon further research, there is a 'Distance(mi)' feature that reflects the mileage of road that was impacted.
    * Furthermore, we can calculate the duration of impact by calculating the delta between 'Start_Time' & 'End_Time'

- 'Description' feature might be an interesting avenue to explore.  However, I suspect that it is collected/generated after the fact so there wouldn't be much predictive value.

- A number of location features
    * 'Start_Lat'/'Start_Lng' are easy enough to interpret but not sure what is meant by 'End_Lat' and 'End_Lng'
    * Similar question for 'Distance(mi)'
    * Street and City might not be useful features since they are not informative without further context.  

- Identifiers in ID and Source

- We have three time features ('Weather_Timestamp', 'Start_Time' and 'End_Time').
    * They need to be [converted to datetimes](#datetimes).  Right now they are just strings.
    * There may be an issue with inconsistent formatting.
    * It may be interesting to use these values to engineer some features like time of day, season, weekday/weekend, etc.
    * However, the information contained in 'Weather_Timestamp' is unclear.

- There are 9 (not including 'Weather_Timestamp') features on the weather.
    * Visability is probably the most relevant of these but 'Weather_Condition' might be a fine substitute/summary.

-There are 5 binary features indicating whether or not it was light or dark out at the time of the incident.

- Next are 10 binary features which seem to provide some information about the road infrastructure at the location of the accident.  


In [ ]:
_TARGET_ = ['Severity']
_NUMERICS_ = ['Distance(mi)','Temperature(F)',
              'Wind_Chill(F)','Humidity(%)','Pressure(in)',
              'Visibility(mi)','Wind_Speed(mph)','Precipitation(in)']

## Missing Values

In [ ]:
if _SPARK_:
    missing = data.select(*[F.sum(F.isnull(F.col(c)).cast("int")).alias(c) for c in data.columns])
    print(missing.show(vertical=True))
else:
    print(data.isna().sum().sort_values(ascending=False))

There is a lot of missing data. 

- Most notably, the 'End_Lat'/'End_Lng'.  Will have to confirm but it would appear that observations missing one are also missing the other.  
- There aren't any missing values for 'Severity' as well as basic location data (gps + state/county).
- Only five observations are missing 'Description' which may prove convenient.


In [ ]:
msgno.heatmap(data)

Good to know:
* Confirmed that Latitude/Longitude Data are strictly missing together.

* Weather data seems to be missing together in general but not perfectly missing and not a uniform relationship (Precipitation and Wind Direction for example).

* The day/night variables are strictly missing together.  

## Duplicates

There are no duplicates to deal with:

In [ ]:
if _SPARK_:
    duplicates = int(data.count() - data.dropDuplicates().count())
    duplicates.show()
else:
    print(data.duplicated().sum())

## Outliers

In [ ]:
if _SPARK_:
    pass
else:
    print(pd.DataFrame({c:{z:count_outliers(data[c],z) for z in [3,5,10,15,20]} for c in _NUMERICS_}).T)

The distance feature may have some outlier issues.  However, this may be a function of the feature's nature as almost all incidents will not impact a large swath of the road. 

## High-Level Features

### Nature of Accident

In [ ]:
data['Severity'].value_counts()

In [ ]:
data['Distance(mi)'].value_counts(bins=25).sort_index(ascending=True)

In [ ]:
(data['Distance(mi)']==0).sum()/data.shape[0]

In [ ]:
data[data['Distance(mi)']==0].groupby('Severity').count()

In [ ]:
sns.histplot(data[data['Distance(mi)'] > 0]['Distance(mi)']);

Interesting that we have distances less than 0.  And as expected, the bulk of the data is near or at 0.  

### Time Features

It is required to set the 'format' parameter to "mixed".  (Too many observations to understand why / explore observations setting off the error.)

In [ ]:
start = pd.to_datetime(data['Start_Time'],format='mixed')
end = pd.to_datetime(data['Start_Time'],format='mixed')
time_change = end - start

In [ ]:
time_change.value_counts(bins=25).sort_index(ascending=True)

### Address Features

In [ ]:
data['Country'].value_counts()

We can cut this 'Country' feature entirely from the data going forward.

In [ ]:
data['State'].value_counts(dropna=False,normalize=True)

California seems very high even adjusting for population and area size.

In [ ]:
data['Zipcode'].value_counts(dropna=False,normalize=True).sort_values(ascending=False)

In [ ]:
data['Airport_Code'].value_counts(dropna=False,normalize=True).sort_values(ascending=False)

In [ ]:
data['Street'].value_counts(dropna=False).sort_values(ascending=False).head(25)

### Weather

In [ ]:
data['Weather_Condition'].unique()

In [ ]:
data['Weather_Condition'].value_counts(dropna=False,normalize=True).sort_values(ascending=False).head(25)

In [ ]:
_WeatherData_ = ['Temperature(F)','Wind_Chill(F)',
                 'Humidity(%)','Pressure(in)','Visibility(mi)',
                 'Wind_Direction','Wind_Speed(mph)','Precipitation(in)',
                 ]

### Night/Day Features

There appears to be multiple indicators for day/night that need to be explored.  According to __[Google](https://www.google.com/search?q=different+types+of+twilight&rlz=1C5OZZY_enUS1127US1127&oq=different+types+of+twilight&gs_lcrp=EgZjaHJvbWUyCQgAEEUYORiABDIICAEQABgWGB4yCAgCEAAYFhgeMggIAxAAGBYYHjIICAQQABgWGB4yDQgFEAAYhgMYgAQYigUyDQgGEAAYhgMYgAQYigUyDQgHEAAYhgMYgAQYigUyDQgIEAAYhgMYgAQYigUyCggJEAAYgAQYogTSAQg0NjU3ajFqN6gCALACAA&sourceid=chrome&ie=UTF-8)__, 

In [ ]:
print(pd.crosstab(data['Nautical_Twilight'],data['Astronomical_Twilight']))
print('-'*50)
print(pd.crosstab(data['Nautical_Twilight'],data['Civil_Twilight']))
print('-'*50)
print(pd.crosstab(data['Nautical_Twilight'],data['Sunrise_Sunset']))

In [ ]:
print(pd.crosstab(data['Astronomical_Twilight'],data['Civil_Twilight']))
print('-'*50)
print(pd.crosstab(data['Astronomical_Twilight'],data['Sunrise_Sunset']))

In [ ]:
print(pd.crosstab(data['Civil_Twilight'],data['Sunrise_Sunset']))

### Infrastructure

# 3-Data Processing

Start with a brand new dataset

In [ ]:
del data
if _SPARK_:
    spark.stop()
else:
    pass

In [ ]:
if _SPARK_:
    spark = SparkSession.builder.appName("Accident Data Project").getOrCreate()
    data_clean = spark.read.csv('US_Accidents_March23.csv',header=True,inferSchema=True)
else:
    if _LITE_SWITCH_:
        data_clean = pd.read_csv('US_Accidents_March23_sampled_500k.csv')
    else:
        data_clean = pd.read_csv('US_Accidents_March23.csv')
    

## Missing Data

## Outliers

## Engineer Features

### <a id='datetimes'> Converting dates </a>

In [ ]:
if _SPARK_:
    pass
else:
    data_clean['Start'] = pd.to_datetime(data_clean['Start_Time'],format='mixed')
    data_clean['End'] = pd.to_datetime(data_clean['End_Time'],format='mixed')

### Partition the time data

In [ ]:
if _SPARK_:
    pass
else:
    data_clean['Month'] = data_clean['Start'].dt.month
    data_clean['Year'] = data_clean['Start'].dt.year
    data_clean['DayofMonth'] = data_clean['Start'].dt.day
    data_clean['DayofWeek'] = data_clean['Start'].dt.day_of_week
    data_clean['Quarter'] = data_clean['Start'].dt.quarter
    data_clean['Hour'] = data_clean['Start'].dt.hour
    data_clean['Date'] = data_clean.apply(lambda r: date(r['Year'],r['Month'],r['Day']),axis=1)

In [ ]:
data_clean['Year'].value_counts(normalize=True).sort_index()

Concern that the two bookend years ('16 & '23) have such low proportion of the data.  

In [ ]:
data_clean['Month'].value_counts(normalize=True).sort_index()

Nothing concerning here.

In [ ]:
data_clean['Quarter'].value_counts(normalize=True)

In [ ]:
data_clean['DayofWeek'].value_counts(normalize=True).sort_index()

In [ ]:
data_clean['Hour'].value_counts(normalize=True).sort_index()

### Weekend Indicator

In [ ]:
data_clean['Weekend'] = ((data_clean['DayofWeek'] == 0) | (data_clean['DayofWeek'] == 6))

In [ ]:
print(f'There are {data_clean['Weekend'].sum()} observations which occured over the weekend.')
data_clean.groupby('Weekend')['Severity'].mean()

### Holidays Indicator

Per __[Google](https://www.google.com/search?client=safari&rls=en&q=holidays+with+most+traffic&ie=UTF-8&oe=UTF-8)__, there are a number of holidays in the US with the highest amount of traffic.  (I subbed NYE in for XMas.)

In [ ]:
thanksgivings = [get_thanksgiving_date(y) for y in range(2016,2024)]
memorials = [calculate_memorial_day(y) for y in range(2016,2024)]
labors = [get_labor_day(y) for y in range(2016,2024)]

In [ ]:
july_4 = ((data_clean['Month'] == 7) & (data_clean['DayofMonth'] == 4))
thanksgiving = data_clean['Date'].apply(lambda d:d in thanksgivings)
memorial = data_clean['Date'].apply(lambda d:d in memorials)
labor = data_clean['Date'].apply(lambda d:d in labors)
nye = ((data_clean['Month'] == 12) & (data_clean['DayofMonth'] == 31))

In [ ]:
holidays = (july_4 | thanksgiving | memorial | labor | nye)
data_clean['Holiday'] = holidays

In [ ]:
info = f'''{holidays.sum()} incidents occured on a prime holiday.'''

print(info)

### Rush Hour Indicator

__[Per Google:  ](https://www.google.com/search?client=safari&rls=en&q=rush+hour+typically&ie=UTF-8&oe=UTF-8)__ The morning rush hour begins around 6a, peaking between 7a & 9a, and eases off by 10a.  While afternoon/evening rush hour begins around 3p, peaking between 4p and 6p, and eases off by 7p.

In [ ]:
data_clean['Morning_Rush'] = ((6 > data_clean['DayofWeek']) & (data_clean['DayofWeek'] > 0) & (data_clean['Start'].dt.hour >= 6) & (10 >= data_clean['Start'].dt.hour))

In [ ]:
print(f'Total of {data_clean['Morning_Rush'].sum()} incidents during the morning rush hour')

In [ ]:
data_clean['Evening_Rush'] = ((6 > data_clean['DayofWeek']) & (data_clean['DayofWeek'] > 0) & (data_clean['Start'].dt.hour >= 15) & (19 >= data_clean['Start'].dt.hour))

In [ ]:
print(f'Total of {data_clean['Evening_Rush'].sum()} incidents during the evening rush hour')

In [ ]:
if _SPARK_:
    accdidents_year = data.stat.crosstab('Year','Severity')
else:
    accdidents_year = pd.crosstab(data_clean['Year'],data_clean['Severity'])

    accdidents_year['Total']=accdidents_year.sum(axis=1)
    accdidents_year['Average']=accdidents_year.apply(lambda r:sum(i*r[i] for i in range(1,5))/r['Total'],axis=1)

In [ ]:
accdidents_year

In [ ]:
accidents_calendar = pd.pivot_table(data_clean,columns='Year',index='Month',values='Severity')

In [ ]:
accidents_calendar

Spatial-Based Data

In [ ]:
data_clean.State.value_counts(normalize=True)

In [ ]:
data_clean['Street'].value_counts(dropna=False).sort_values(ascending=False).head(50)

In [ ]:
data_clean["Street"].str.startswith('US').sum()

In [ ]:
data_clean["Street"].str.startswith('United').sum()

In [ ]:
data_clean["Street"].str.startswith('I-').sum()

In [ ]:
data_clean["Street"].str.startswith('Interstate').sum()

In [ ]:
data_clean["Street"].str.startswith('Route').sum()

In [ ]:
data_clean["Street"].str.startswith('Rt').sum()

### Weather Consolidation

### Nature of Incident

In [ ]:
data_clean['Time_of_Impact(hr)'] = (data_clean['End'] - data_clean['Start']).dt.seconds/360

In [ ]:
sns.histplot(data_clean['Time_of_Impact(hr)'])

## Saveout Data

In [ ]:
if _LITE_SWITCH_:
    data_clean.to_csv('AccidentData_Sampled_Clean.csv')
else:
    data_clean.to_csv('AccidentData_Clean.csv')

# 4-Full EDA

## Nature of Incident Variables

In [ ]:
data_clean.groupby('Severity')[['Distance(mi)','Time_of_Impact(hr)']].describe().T

In [ ]:
if _LITE_SWITCH_:
    sns.histplot(data_clean['Severity'])
else:
    pass

In [ ]:
palette = {1:'red',2:'blue',3:'green',4:'black'}
sns.pairplot(data_clean[['Severity','Distance(mi)','Time_of_Impact(hr)']],hue='Severity',palette=palette);

## Feature Variables

### Relationships

In [ ]:
if _SPARK_:
    corr_data = data_clean.select(_NUMERICS_)
    col_names = corr_data.columns
    features = corr_data.rdd.map(lambda row: row[0:]) 
    corrs = Statistics.corr(features, method="pearson")
else:
    corrs = data_clean[_NUMERICS_].corr()
    print(corrs)
   

In [ ]:
sns.heatmap(corrs,vmin=-1,vmax=1,cmap='coolwarm')

### Against Target Variable

### Infrastructure

### Weather Conditions

In [ ]:
conditions = ['Temperature(F)','Wind_Chill(F)','Humidity(%)',
              'Pressure(in)','Visibility(mi)',
              'Wind_Speed(mph)','Precipitation(in)'
              ]

In [ ]:
X = data_clean[conditions].dropna()
X_std = StandardScaler().fit_transform(X)

In [ ]:
pca = PCA()
pca.fit(X_std);

In [ ]:
cumulative_var = np.cumsum(pca.explained_variance_ratio_)
# Create a cumulative variance plot
plt.figure(figsize=(10, 6))
cum_var_plot = plt.plot(range(1, len(cumulative_var) + 1), cumulative_var, 
         'o-', linewidth=2, color='green')
# Add lines for 90% and 95% thresholds
plt.axhline(y=0.9, color='r', linestyle='--', label='90% threshold')
plt.axhline(y=0.95, color='g', linestyle='--', label='95% threshold')
plt.title('Cumulative Variance Explained')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Variance Explained')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
target_variance = 0.95
n_components_variance = np.argmax(cumulative_var >= target_variance) + 1
print(f'We choose to include the first {n_components_variance} components.')

In [ ]:
pca = PCA(n_components=n_components_variance)
pca.fit(X_std);
# Transform the standardized data to get principal components
X_pca = pca.transform(X_std)

# Create a DataFrame with the principal components
pca_df = pd.DataFrame(
    data=X_pca,
    columns=[f'PC{i+1}' for i in range(X_pca.shape[1])]
)
pca_df

In [ ]:
# Create an easy to view DF of the loadings.
loadings = pd.DataFrame(
    pca.components_.T,
    columns=[f'PC{i+1}' for i in range(n_components_variance)],
    index=conditions
)
loadings

In [ ]:
# Visualize the loadings for the first two PCs
plt.figure(figsize=(12, 8))
for i, feature in enumerate(conditions):
    plt.arrow(0, 0, loadings.iloc[i, 0], loadings.iloc[i, 1], head_width=0.05, head_length=0.05)
    plt.text(loadings.iloc[i, 0]*1.1, loadings.iloc[i, 1]*1.1, feature, fontsize=12)

# Add a unit circle for reference
circle = plt.Circle((0, 0), 1, fill=False, linestyle='--')
plt.gca().add_patch(circle)

plt.grid(True)
plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
plt.axvline(x=0, color='k', linestyle='-', alpha=0.3)
plt.xlim(-1.1, 1.1)
plt.ylim(-1.1, 1.1)
plt.title('PCA Loading Plot (PC1 vs PC2)')
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance explained)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance explained)')
plt.tight_layout()
plt.show()

In [ ]:
X = data_clean[conditions].dropna()
X_std = StandardScaler().fit_transform(X)

In [ ]:
# Write your code here
km = KMeans(random_state=42)
visualizer = KElbowVisualizer(km,k=(5,50))
visualizer.fit(X_std)
visualizer.show();

# 5-Statistical Analysis
Chi-Squared and ANOVA Tests for various features 

## Severity as An Input Feature for Distance and Time of Impact

In [ ]:
model_weather = ols("Q('Distance(mi)') ~ C(Severity)", data=data_clean).fit()
anova_table_weather = sm.stats.anova_lm(model_weather, typ=2)
print("\nANOVA for Weather Category:\n", anova_table_weather)
p_weather = anova_table_weather['PR(>F)'].iloc[0]
print(f"P-value for Weather Category: {p_weather}")

In [ ]:
model_weather = ols("Q('Time_of_Impact(hr)') ~ C(Severity)", data=data_clean).fit()
anova_table_weather = sm.stats.anova_lm(model_weather, typ=2)
print("\nANOVA for Weather Category:\n", anova_table_weather)
p_weather = anova_table_weather['PR(>F)'].iloc[0]
print(f"P-value for Weather Category: {p_weather}")

## Weather Category
* Null Hypothesis (H0): There is no significant difference in accident severity across weather categories.

* Alternative Hypothesis (H1): There is a significant difference in accident severity across weather categories.

In [ ]:
def test_category(feature_name):
    # Time of Impact--ANOVA
    model_time = ols(f"Q('Time_of_Impact(hr)') ~ C({feature_name})", data=data_clean).fit()
    anova_table_time = sm.stats.anova_lm(model_weather, typ=2)
    # Distance of Impact--ANOVA
    model_distance = ols(f"Q('Distance(mi)') ~ C({feature_name})", data=data_clean).fit()
    anova_table_distance = sm.stats.anova_lm(model_weather, typ=2)
    # Severity Distribution--ANOVA
    contingency_table = pd.crosstab(data_clean['Weather_Condition'], data_clean['Severity'])
    chi_results = chi2_contingency(contingency_table)
    return anova_table_time, anova_table_distance, chi_results

In [ ]:
model_weather = ols("Q('Time_of_Impact(hr)') ~ C(Weather_Condition)", data=data_clean).fit()
anova_table_weather = sm.stats.anova_lm(model_weather, typ=2)
print("\nANOVA for Weather Category:\n", anova_table_weather)
p_weather = anova_table_weather['PR(>F)'].iloc[0]
print(f"P-value for Weather Category: {p_weather}")

In [ ]:
model_weather = ols("Q('Distance(mi)') ~ C(Weather_Condition)", data=data_clean).fit()
anova_table_weather = sm.stats.anova_lm(model_weather, typ=2)
print("\nANOVA for Weather Category:\n", anova_table_weather)
p_weather = anova_table_weather['PR(>F)'].iloc[0]
print(f"P-value for Weather Category: {p_weather}")

In [ ]:
# Create a contingency table of two categorical variables
contingency_table = pd.crosstab(data_clean['Weather_Condition'], data_clean['Severity'])
# Perform the Chi-squared test
chi2, p, dof, expected = chi2_contingency(contingency_table)
print(f"Chi-squared statistic: {chi2}")
print(f"P-value: {p}")
print(f"Degrees of freedom: {dof}")
#print("Expected frequencies table:")
#print(expected)

## Advanced Analysis

# 6-Insights & Conclusions

## Annual Distinctions

Number of Accidents; Distribution of Severities; Length of Impact

## Temporal & Spatial Considerations

How do accident counts relate to different times of the day and for different region types (urban vs rural)?

## Weather Considerations
Does certain weather conditions produce more severe incidents?  

## Severity Prediction

## Unusual Weather 

Does driving in unexpected weather--based on area, time of year and/or both--create a higher likelihood of an accident.

## Famous Highways

## Highway's Near Urban Areas

For cross-state

## Naturual Language

Examination of the description feature.

## Safety Infrastructure
Does certain road infrastructure projects help reduce the number of incidents?

## New Traffic Pattern

Does the existence of a new traffic pattern in the area increase the likelihood of an accident?

## Recent Accident Indicator

Does the presence of one accident, predict another.